In [1]:
import numpy as np
import codecs
import math
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dropout, Flatten, Dense
from keras import applications

# dimensions of our images.
img_width, img_height = 150, 150

top_model_weights_path = 'model_with_pre-train.h5'
train_data_dir = 'data/train'
validation_data_dir = 'data/validation'
nb_train_samples = 2500
nb_validation_samples = 1000
epochs = 30
batch_size = 16

Using Theano backend.
Can not use cuDNN on context None: Device not supported
Mapped name None to device cuda: GeForce 710M (0000:01:00.0)


In [2]:
datagen = ImageDataGenerator(rescale=1. / 255)

# build the VGG16 network
model = applications.VGG16(include_top=False, weights='imagenet')

generator = datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode=None,
    shuffle=False)
features_train = model.predict_generator(
    generator, math.ceil(nb_train_samples / batch_size))
np.save(open('features_train.npy', 'wb'),
    features_train)

generator = datagen.flow_from_directory(
    validation_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode=None,
    shuffle=False)
features_validation = model.predict_generator(
    generator, math.ceil(nb_validation_samples / batch_size))
np.save(open('features_validation.npy', 'wb'),
    features_validation)

Found 2500 images belonging to 5 classes.
Found 1000 images belonging to 5 classes.


In [3]:
train_data = np.load(open("features_train.npy","rb"))
train_labels = np.array([0] * int(nb_train_samples / 2) + [1] * int(nb_train_samples / 2))

validation_data = np.load(open("features_validation.npy","rb"))
validation_labels = np.array([0] * int(nb_validation_samples / 2) + [1] * int(nb_validation_samples / 2))

In [4]:
model = Sequential()
model.add(Flatten(input_shape=train_data.shape[1:]))
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))

model.compile(optimizer='rmsprop',
    loss='binary_crossentropy', metrics=['accuracy'])

In [ ]:
model.fit(train_data, train_labels,
    epochs=epochs,
    batch_size=batch_size,
    validation_data=(validation_data, validation_labels))


Train on 2500 samples, validate on 1000 samples
Epoch 1/30
2500/2500 [==============================] - 2s - loss: 0.0413 - acc: 0.9896 - val_loss: 0.5267 - val_acc: 0.9340

In [8]:
model.save_weights(top_model_weights_path)
scores = model.evaluate(validation_data, validation_labels, verbose=0)
print("Точность работы на тестовых данных: %.2f%%" % (scores[1]*100))

Точность работы на тестовых данных: 92.90%
